### A/B Hypothesis Testing
    Accept or reject the following Null Hypotheses: 
        There are no risk differences across provinces 
        There are no risk differences between zip codes 
        There are no significant margin (profit) difference between zip codes 
        There are not significant risk difference between Women and Men


### There are no risk differences across provinces 

In [1]:
import os 
os.chdir("../")

In [2]:
import pandas as pd
df = pd.read_csv("data/dataset.csv")

C:\Users\windows 10\AppData\Local\Temp\ipykernel_4640\421909392.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data.txt", delimiter="|")


In [3]:
df.to_csv('data/dataset.csv')

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Load your dataset
data = pd.read_csv("your_dataset.csv")

# Create contingency table
contingency_table = pd.crosstab(data["province"], data["high_risk"])

# Perform chi-squared test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Analyze p-value
if p_value < 0.05:
    print("Reject null hypothesis: There are significant risk differences across provinces.")
else:
    print("Fail to reject null hypothesis: There are no significant risk differences across provinces.")